# SAR IMAGE CLASSIFICATION
- Classification problem
- Oceanography/Meteorology
- Pierre Tandeo (pierre.tandeo@imt-atlantique.fr), Ronan Fablet (ronan.fablet@imt-atlantique.fr), Carlos Granero (carlos.granero-belinchon@imt-atlantique.fr)

## Data and aim

In this project, we use SAR (Synthetic Aperture Radar) images (20km x 20km) of the ocean surface. The goal is to automatically classify different oceanic and atmospheric phenomena. Below, we will find an example of 10 classes corresponding to 10 different phenomena (among pure ocean swell in F, convective cells in I, icebergs in L, etc...):

![SAR](https://tandeo.files.wordpress.com/2019/01/sar_classes.png)

Here, we will use a database of 160 SAR images per class, anotated by experts. Data were collected in 2016 by the Sentinel-1 satellite.

## Evaluation and benchmark

We divided the database in two parts: "training" and "validation" folders with respectively 70% and 30% of the dataset. To learn the model (here a classifier), we propose to use cross-validations on the training dataset. Then, you will use the validation dataset to evaluate the performance of your model, using the total pourcentage of well predicted data. The reference result is **94% of accuracy** and is given by a deep leaning architecture. The reference paper is given below.

## Reference
- https://tandeo.files.wordpress.com/2019/11/wang_2019.pdf

### Import libraries

In [1]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install imageio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Prepare the datasets

Now, we load the training (70%) and validation (30%) datasets.

In [4]:
!wget -O SAR.zip https://www.dropbox.com/s/c1vkkkxo33efpke/SAR.zip?dl=0

--2023-06-27 13:25:19--  https://www.dropbox.com/s/c1vkkkxo33efpke/SAR.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/c1vkkkxo33efpke/SAR.zip [following]
--2023-06-27 13:25:20--  https://www.dropbox.com/s/raw/c1vkkkxo33efpke/SAR.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc793a46f1870571c3bb5ca55069.dl.dropboxusercontent.com/cd/0/inline/B-zJz-imcQq2461e5CaQUZXMnWG0wei3xuQ2LB8sEGgpVrb6_RJrO6z0HxQ3F1Ls2btat1Q2OBcf7RnJsNRaJsKQyYp3LTV-B1OEceJQWcHBRYNeAAlKrggHJEqIUQ654WaIqSKT18C6PbzEJBuw3zcJDTVTW3MPC6Q0ovjnmYpcLA/file# [following]
--2023-06-27 13:25:20--  https://uc793a46f1870571c3bb5ca55069.dl.dropboxusercontent.com/cd/0/inline/B-zJz-imcQq2461e5CaQUZXMnWG0wei3xuQ2LB8sEGgpVrb6_RJrO6z0HxQ3F1Ls2btat1Q2OBcf7RnJsNRaJsKQyY

In [5]:
!unzip SAR.zip
# We remove the zip file
!rm SAR.zip

Archive:  SAR.zip
   creating: training/
   creating: training/F/
  inflating: training/F/s1a-wv1-slc-vv-20160210t150613-20160210t150616-009885-00e7d4-003.jpg  
  inflating: training/F/s1a-wv1-slc-vv-20160214t222921-20160214t222924-009948-00e9bc-047.jpg  
  inflating: training/F/s1a-wv1-slc-vv-20160215t102110-20160215t102113-009955-00e9f1-013.jpg  
  inflating: training/F/s1a-wv1-slc-vv-20160215t181529-20160215t181532-009960-00ea14-121.jpg  
  inflating: training/F/s1a-wv1-slc-vv-20160218t141037-20160218t141040-010001-00eb4f-045.jpg  
  inflating: training/F/s1a-wv1-slc-vv-20160219t162145-20160219t162148-010017-00ebc3-009.jpg  
  inflating: training/F/s1a-wv1-slc-vv-20160227t114202-20160227t114205-010131-00ef0a-075.jpg  
  inflating: training/F/s1a-wv1-slc-vv-20160301t233725-20160301t233728-010182-00f06c-067.jpg  
  inflating: training/F/s1a-wv1-slc-vv-20160304t044928-20160304t044931-010214-00f154-075.jpg  
  inflating: training/F/s1a-wv1-slc-vv-20160311t110045-20160311t110048-010320-0

In [10]:
# import functions
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from imageio.v2 import imread
import sklearn

# paths to the training and validation datasets
path_train = "training/"
path_validation = "validation/"

# class names
classes = ['F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']

# initialization
X_train = []
y_train = []
X_validation = []
y_validation = []

# loop on images
for j in range(len(classes)):
    path = path_train + classes[j]
    files = [f for f in listdir(path) if isfile(join(path, f))]

    # training
    for i in range(len(files)):
        tmp = imread(path + '/' + files[i])
        X_train.append(tmp[0:450,0:450:,0])
        y_train.append(classes[j])

    # validation
    path = path_validation + classes[j]
    files = [f for f in listdir(path) if isfile(join(path, f))]
    for i in range(len(files)):
        tmp = imread(path + '/' + files[i])
        X_validation.append(tmp[0:450,0:450:,0])
        y_validation.append(classes[j])

# transform to array
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_validation = np.asarray(X_validation)
y_validation = np.asarray(y_validation)